In [95]:
import pandas as pd
import numpy as np

In [96]:
# Lê o arquivo CSV
tabela = pd.read_csv('diabetes_prediction_dataset.csv')
# Mostra as 5 primeiras linhas da tabela
tabela.head()


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


Ao analizarmos as primeiras linhas da tabela de dados que utilizaremos para treinar nosso modelo, nos deparamos com duas colunas que requerem especial atenção: gender e smoking_history.
Ambas as colunas possuem valores não númericos e portanto devem ser tratadas antes da utilização.
A coluna 'gender' pode ser tratada de maneira bem simples, como os possíveis valores são apenas dois (Male ou Female), podem facilmente ser mapeados para valores binários.

In [97]:
# Converte as informações da coluna gênero para um valor binário
tabela['gender'] = tabela['gender'].map({'Female': 1, 'Male': 0})

tabela.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,1.0,80.0,0,1,never,25.19,6.6,140,0
1,1.0,54.0,0,0,No Info,27.32,6.6,80,0
2,0.0,28.0,0,0,never,27.32,5.7,158,0
3,1.0,36.0,0,0,current,23.45,5.0,155,0
4,0.0,76.0,1,1,current,20.14,4.8,155,0


O segundo caso exige um pouco de maior cautela, a gama de valores presentes na coluna 'smoking_history' é maior, não é possível mapeá-los para valores binários.
Os possíveis valores presentes nesta coluna são: never, former, current, not current e No Info.
Todos estes valores, com exceção do último, devem ser mapeados para valores númericos.

In [98]:
# Converte informações na coluna smoking_history
tabela['smoking_history'] = tabela['smoking_history'].map({'never': 0, 'former': 1, 'current': 2, 'not current': 3})
tabela.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,1.0,80.0,0,1,0.0,25.19,6.6,140,0
1,1.0,54.0,0,0,NaN,27.32,6.6,80,0
2,0.0,28.0,0,0,0.0,27.32,5.7,158,0
3,1.0,36.0,0,0,2.0,23.45,5.0,155,0
4,0.0,76.0,1,1,2.0,20.14,4.8,155,0


In [99]:
# Conta ocorrencias de NaN na coluna smoking_history
tabela['smoking_history'].isna().sum()

np.int64(39820)

Como existem 39820 células sem informação na coluna 'smoking_history', não podemos simplesmente ignorar tais linhas, seria uma perda expressiva de nossa base de dados.

In [100]:
tabela['gender'].isna().sum()

np.int64(18)

Na coluna 'gender' por outro lado, existem apenas 18 linhas não preenchidas e portanto, seu descarte não apresenta um impacto consideravel.

In [101]:
tabela = tabela[~tabela['gender'].isna()]
tabela['gender'].isna().sum()

np.int64(0)

Voltando à coluna 'smoking_history', as informações contidas nessa coluna são de extrema relevância para o treinamento do modelo, visto que, o histórico em tabagismo aumenta as chances de desenvolver diabetes tipo 2. Portanto, a coluna não pode ser descartada por inteiro. Além disso, existem muitos valores não preenchidos, descartar tais linhas geraria uma enorme perda em nossos dados. Nos resta então apenas uma opção: preencher os dados ausentes da coluna.

A estratégia de preenchimento adotada será a utilização da média de todos os valores existentes na coluna.

In [102]:
media_tabagismo = tabela['smoking_history'].mean()
tabela['smoking_history'] = tabela['smoking_history'].fillna(media_tabagismo)
tabela['smoking_history'].isna().sum()

np.int64(0)

In [103]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer

    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """

    np.random.seed()

    W1 = np.random.randn(n_h, n_x) * 0.001
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.001
    b2 = np.zeros((n_y, 1))

    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [104]:
def feed_forward(X, parameters):
    """
    Argument:
    X -- dados de input de tamanho  (n_x, m)
    parameters -- Dicionario python com nossos parametros (saída da função de inicialização)

    Returns:
    A2 -- A saída sigmóide da segunda ativação
    cache -- um dicionário contendo "Z1", "A1", "Z2" e "A2"
    """

    Z1 = np.dot(parameters["W1"], X) + parameters["b1"]
    A1 = np.tanh(Z1)
    Z2 = np.dot(parameters["W2"], A1) + parameters["b2"]
    A2 = sigmoid(Z2)

    assert (A2.shape == (1, X.shape[1]))

    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}

    return A2, cache

In [118]:
def compute_cost(A2, Y, parameters):
    """
    Calcula o custo de entropia cruzada dado na equação (13)

    Arguments:
    A2 - A saída sigmóide da segunda ativação, de formato (1, número de observações)
    Y - vetor de forma de rótulos "verdadeiros" (1, número de observações)
    parâmetros - dicionário python contendo seus parâmetros W1, b1, W2 e b2

    Returns:
    cost -- custo de entropia cruzada dada a equação (13)
    """

    m = Y.shape[1]

    logprobs = np.multiply(np.log(A2), Y)
    cost = - np.sum(logprobs)

    cost = np.squeeze(cost)
    assert (isinstance(cost, float))

    return cost

In [106]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implemente a backward propagation usando as instruções acima.

    Arguments:
     parameters - dicionário python contendo os parâmetros
     cache -- um dicionário contendo "Z1", "A1", "Z2" e "A2".
     X – dados de entrada da forma (2, número de observações)
     Y - vetor de forma de rótulos "verdadeiros" (1, número de observações)

    Returns:
    grads -- dicionário python contendo seus gradientes em relação a diferentes parâmetros
    """
    m = X.shape[1]

    dZ2 = cache["A2"] - Y
    dW2 = (1 / m) * np.dot(dZ2, cache["A1"].T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(parameters["W2"].T, dZ2) * (1 - np.power(cache["A1"], 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}

    return grads

In [107]:
def layer_sizes(X, Y):
    """
    Arguments:
    X – conjunto de dados de entrada de forma (tamanho de entrada, número de observações)
    Y – rótulos de forma (tamanho de saída, número de observações)

    Returns:
    n_x – o tamanho da camada de entrada
    n_h – o tamanho da camada oculta
    n_y – o tamanho da camada de saída
    """
    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]
    return (n_x, n_h, n_y)

In [108]:
def update_parameters(parameters, grads, learning_rate=1.2):
    """
    Atualiza os parâmetros usando a regra de atualização de gradiente descendente

    Arguments:
    parameters – dicionário python contendo seus parâmetros
    grads – dicionário python contendo seus gradientes

    Returns:
    parameters – dicionário python contendo seus parâmetros atualizados
    """
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    W1 = W1 - (learning_rate * grads["dW1"])
    b1 = b1 - (learning_rate * grads["db1"])
    W2 = W2 - (learning_rate * grads["dW2"])
    b2 = b2 - (learning_rate * grads["db2"])

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [109]:
def sigmoid(x):
    """
    Calcular o sigmóide de x

    Arguments:
    x -- Uma matriz escalar ou numpy de qualquer tamanho.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

In [128]:
def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):
    """
    Arguments:
    X - conjunto de dados de forma (categorias, número de observações)
    Y - rótulos de forma (1, número de observações)
    n_h – tamanho da camada oculta
    num_iterations -- Número de iterações no loop de descida gradiente
    print_cost -- se True, imprime o custo a cada 1000 iterações

    Returns:
    parameters -- parâmetros aprendidos pelo modelo. Eles podem então ser usados para prever.
    """

    np.random.seed()
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]

    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    for i in range(0, num_iterations):
        A2, cache = feed_forward(X, parameters)

        cost = compute_cost(A2, Y, parameters)

        grads = backward_propagation(parameters, cache, X, Y)

        parameters = update_parameters(parameters, grads, learning_rate=1.2)

        if print_cost and i % 1000 == 0:
            print("Custo após a interação %i: %f" % (i, cost))

    return parameters

In [113]:
# Separa variáveis independentes e dependentes da tabela
Y = tabela['diabetes'].to_frame().to_numpy().T
X = tabela.drop(columns='diabetes').to_numpy().T
print(X.shape)
print(Y.shape)

(8, 99982)
(1, 99982)


In [129]:
parameters = nn_model(X, Y, 4, num_iterations=1001, print_cost=True)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

Custo após a interação 0: 5892.106135
Custo após a interação 1000: 20951.857145
W1 = [[-8.76707304e-04 -1.27093110e-01 -1.03069479e-03  9.99177816e-04
  -1.29965190e-03 -9.15211070e-02 -1.76664397e-02 -3.81080847e-01]
 [ 1.57822002e-05  2.02040472e-02 -1.63376529e-03  1.62146882e-03
  -5.57654784e-04  1.80854870e-02  3.30330965e-03  6.94273387e-02]
 [-1.21583797e-04  6.81584357e-02 -8.28142725e-04  1.73798464e-04
   2.34316817e-03  4.84149356e-02  9.60378164e-03  2.08796205e-01]
 [ 1.22819665e-03  2.03913395e-02  1.91468536e-05  1.65574798e-04
   8.75051823e-04  1.63405327e-02  2.71252332e-03  6.92724647e-02]]
b1 = [[-0.00339286]
 [ 0.00067869]
 [ 0.00185228]
 [ 0.00070944]]
W2 = [[ 0.01009931 -0.66685769  0.00674351 -0.60213728]]
b2 = [[-1.1037257]]
